In [1]:
'''
Info.

전역변수들
1. 역과 주변역 정보 및 시간을 저장할 stations 변수
stations = {'역명': {'주변역' : 시간, '주변역' : 시간}, '역명' : {'주변역' : 시간, '주변역' : 시간}, ......}


'''

"\nInfo.\n\n전역변수들\n1. 역과 주변역 정보 및 시간을 저장할 stations 변수\nstations = {'역명': {'주변역' : 시간, '주변역' : 시간}, '역명' : {'주변역' : 시간, '주변역' : 시간}, ......}\n\n\n"

In [2]:
import json

with open('stations.json', encoding='utf-8-sig')as json_file:  #Open Saved Json file to Dictionary
    stations = json.load(json_file)
with open('stations_info.json', encoding='utf-8-sig')as json_file:  #Open Saved Json file to Dictionary
    stations_info = json.load(json_file)
with open('transfer_data.json', encoding='utf-8-sig')as json_file:  #Open Saved Json file to Dictionary
    transfer_data = json.load(json_file)

In [3]:
from collections import deque

In [4]:
def getLine(st1, st2):
    line1 = stations_info[st1]
    line2 = stations_info[st2]
    
    link = []
    for num1 in line1:
        for num2 in line2:
            if num1 == num2:
                link.append(num1)
    return link

In [20]:
def backtracking(line, station, n, time, visited = {}): #start station은 line input을 ""로(empty string)
    if time > n: return #소요시간이 n분을 넘어갈 경우 해당 역은 탐색하지 않는다
    
    around_stations = stations[station]
    
    if station in visited:
        if visited[station] > time:
            visited[station] = time
        else:
            return
    else:
        visited[station] = time
    
    for st_name, takes_time in around_stations.items():#주변 역들 탐색
        next_time = time + takes_time #다음 역까지 걸린 시간
        link = getLine(station, st_name)
        for riding in link:
            if line == "": #출발역 예외처리
                backtracking(riding, st_name, n, next_time, visited)
            if riding == line:
                backtracking(riding, st_name, n, next_time + 0.5, visited) #정차시간 추가
            else:
                try:
                    transfer_time = transfer_data[station][line][riding]
                except KeyError:
                    continue
                backtracking(riding, st_name, n, next_time + 3.0 + transfer_time, visited) #정차시간 + 환승시간 + 대기시간
    st_list = list(visited.keys())
    return st_list

In [21]:
def bfs(station, n, time, visited = []):
    queue = deque([(station, time)])
    while queue:
        st_info = queue.popleft()
        st_name = st_info[0]
        elapsed_time = st_info[1]
        
        try:
            around_stations = stations[st_name]
        except KeyError:
            continue
        visited.append(st_name)
        
        for next_name, takes_time in around_stations.items():
            if next_name in visited:
                continue
            next_time = elapsed_time + takes_time + 0.5 #0.5=정차시간
            if next_time <= n:
                queue.append((next_name, next_time))
    visited = set(visited)
    return visited
        

In [22]:
def getInput():
    n = int(input('사람이 몇 명입니까? '))
    station_list = []
    #n = 4
    #station_list = ['충무로', '선정릉', '잠실', '건대입구']
    #n = 1
    #station_list = ['잠실']
    
    for i in range(n):
        temp = input('탑승 역을 입력하시오: ')
        station_list.append(temp)
        
    return station_list, n

In [23]:
def binarySearch(accessible_list, station):
    size = len(accessible_list)
    left = 0
    right = size - 1
    
    while left <= right:
        mid = int((left + right) / 2)
        if accessible_list[mid] == station:
            return True
        elif accessible_list[mid] > station:
            right = mid - 1
        else:
            left = mid + 1
    return False

In [24]:
def getList(accessible_list, n): #갈수있는 역들 모음(2차원행렬), 사람 수
    duplicate_list = []
    for station in accessible_list[0]:
        duplicated = True
        for i in range(1, n):
            duplicated = duplicated and binarySearch(accessible_list[i], station)
        if duplicated:
            duplicate_list.append(station)
    return duplicate_list

In [25]:
def main():
    station_list, n = getInput()
    #accessible_list = []
    
    while True:
        accessible_list = []
        time = int(input("input time: "))
        if time == -1: break
    
        for i in range(n):
            possible_station = backtracking("", station_list[i], time, 0.0, {})
            #print("=================================")
            possible_station = sorted(possible_station)
            accessible_list.append(possible_station)
    
        #print(accessible_list)
        dest_stations = []
        dest_stations = getList(accessible_list, n)

        print(dest_stations)
        #print(accessible_list)
        #for i in range(len(accessible_list[0]) - 1):
        #    print(accessible_list[0][i] < accessible_list[0][i+1])

In [28]:
if __name__ == '__main__':
    main()

사람이 몇 명입니까? 4
탑승 역을 입력하시오: 충무로
탑승 역을 입력하시오: 건대입구
탑승 역을 입력하시오: 선정릉
탑승 역을 입력하시오: 잠실
input time: 13
['왕십리']
input time: 14
['왕십리']
input time: 15
['건대입구', '왕십리']
input time: 16
['건대입구', '왕십리']
input time: 17
['건대입구', '고속터미널', '왕십리']
input time: 18
['건대입구', '고속터미널', '교대', '구의', '군자', '마장', '상왕십리', '성수', '왕십리', '한양대', '행당']
input time: 19
['강변', '건대입구', '고속터미널', '교대', '구의', '군자', '답십리', '뚝섬', '마장', '상왕십리', '성수', '신당', '왕십리', '잠원', '한양대', '행당']
input time: 20
['강남', '강변', '건대입구', '고속터미널', '교대', '구의', '군자', '남부터미널', '답십리', '뚝섬', '뚝섬유원지', '마장', '사평', '상왕십리', '성수', '신금호', '신당', '신사', '어린이대공원', '왕십리', '응봉', '잠원', '한양대', '행당']
input time: -1


 *송민수의 아이디어*

4개의 기준 역이 있다고 칠 때
원래는 1번째로 나오는 배열의 역 잡고 for문으로 다 돌리면서 각 역마다 (정렬해놓고) 이진탐색을 돌릴까 했는데
이걸 반으로 나눠서 1,3번째 배열에서 기준을 각각 잡아서 for문 돌리고 divide and conquer방식으로 점점 합쳐가면 된다.
